Finds closest natural protein and prepares files for ddGan

In [19]:
trainpath = "C:/Users/micha/Downloads/spikeprot_train.txt"
validpath = "C:/Users/micha/Downloads/spikeprot_valid.txt"
testpath = "C:/Users/micha/Downloads/spikeprot_test.txt"
with open(trainpath) as f:
    train = f.read().splitlines()
with open(validpath) as f:
    valid = f.read().splitlines()
with open(testpath) as f:
    test = f.read().splitlines()

all_prot = []
all_prot.extend(train)
all_prot.extend(valid)
all_prot.extend(test)

del train
del valid
del test

all_prot = list(map(lambda x: x.replace("-",""), all_prot))
all_prot_np = np.vstack(list(map(lambda x: np.array(list(x)),all_prot)))


In [ ]:
import Bio 
from Bio.pairwise2 import format_alignment
# Based on from https://github.com/hcgasser/SpikeVAE/blob/main/evaluation/get_point_mutation_list.py
def generate_point_mutation_list(original_sequence, mutated_sequence):
    point_mutation_list = ""
    for res_index, residue_tuple in enumerate(zip(original_sequence, mutated_sequence)):
        orig_res = residue_tuple[0]
        mut_res = residue_tuple[1]
        if mut_res == "-" or orig_res == "-":
            # potentially an insertion or deletion -ignore
            continue
        elif mut_res != orig_res:
            # a mutation - add it to the list
            point_mutation_list+=f"{orig_res}{res_index+1}{mut_res}\n"
        else:
            # no mutation - ignore
            continue

    return point_mutation_list

In [42]:
import numpy as np
result = []
with open("transformer_to_evaluate.txt") as f:
    for line in f:
        to_eval.append(line)
        arg_max = np.argmax(np.sum(all_prot_np == np.array(list(line.strip())),axis = 1))
        result.append(all_prot[arg_max])
        

In [47]:
import os
newpath = "ddgan/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

for i in range(len(to_eval)):
    mutated = to_eval[i]
    original = result[i]
    alignments = Bio.pairwise2.align.globalxs(original, mutated, score_only=False, open=-1, extend=-0.1)
    f = open(f"ddgan/seq{i}.mut", "a")
    f.write(generate_point_mutation_list(alignments[0].seqA, alignments[0].seqB))
    f.close()
    f = open(f"ddgan/seq{i}.fasta", "a")
    f.write(f">{i}\n"+alignments[0].seqA.replace('-','X'))
    f.close()

In [84]:
np.save("trans_closest.npy",np.array(result))